In [1]:
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import mnist_train
import mnist_inference

C:\Users\Philip\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
EVAL_INTERVAL_SECS = 10

def evaluate(mnist):
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32, [None, mnist_inference.INPUT_NODE], name="x-input")
        y = tf.placeholder(tf.float32, [None, mnist_inference.OUTPUT_NODE], name="y-input")
        
        validata_feed = {x: mnist.validation.images,
                         y: mnist.validation.labels}
        
        y_ = mnist_inference.inference(x, None)
        
        correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        #變量重命名來加載模型，就可以調用mnist_inference.inference的前向傳播來計算移動平均了
        variable_averages = tf.train.ExponentialMovingAverage(mnist_train.MOVING_AVERAGE_DECAY)
        variable_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variable_to_restore)
    
        while True:
            with tf.Session() as sess:
                ckpt = tf.train.get_checkpoint_state(mnist_train.MODEL_SAVE_PATH)
                
                if (ckpt and ckpt.model_checkpoint_path):
                    saver.restore(sess, ckpt.model_checkpoint_path)
                    
                    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('/')[-1]
                    
                    accuracy_score = sess.run(accuracy, feed_dict = validata_feed)
                    
                    print("After %s training step(s), validation accuracy = %g" % (global_step, accuracy_score))
                    
                else:
                    print("No checkpoint file found")
                    return
                
            time.sleep(EVAL_INTERVAL_SECS)


In [5]:
def main(argv=None):
    mnist = input_data.read_data_sets("./MNIST DATA", one_hot=True)
    evaluate(mnist)

if __name__ == '__main__':
    main()

Extracting ./MNIST DATA\train-images-idx3-ubyte.gz
Extracting ./MNIST DATA\train-labels-idx1-ubyte.gz
Extracting ./MNIST DATA\t10k-images-idx3-ubyte.gz
Extracting ./MNIST DATA\t10k-labels-idx1-ubyte.gz
No checkpoint file found
